In [1]:
print('hi')

hi


In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf

In [3]:
conf = SparkConf().setAppName("optimisation").setMaster("local[*]")
sc = SparkContext(conf = conf)

24/04/10 19:15:31 WARN Utils: Your hostname, arphaxad-HP-Notebook resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlp3s0)
24/04/10 19:15:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/10 19:15:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
#group by key
data_sample = [(1,4),(2,2),(2,1),(3,5),(2,5),(2,10),(2,7),(3,4),(2,1),(2,4),(4,4)]
rdd_sample = sc.parallelize(data_sample,3)
rdd_sample.glom().collect()
rdd_sample.getNumPartitions()

3

In [8]:
# rdd_sample.collect()


[(1, 4),
 (2, 2),
 (2, 1),
 (3, 5),
 (2, 5),
 (2, 10),
 (2, 7),
 (3, 4),
 (2, 1),
 (2, 4),
 (4, 4)]

In [13]:
sc.defaultParallelism

4

In [14]:
rdd_sample.glom().collect()

[[(1, 4), (2, 2), (2, 1)],
 [(3, 5), (2, 5), (2, 10)],
 [(2, 7), (3, 4), (2, 1), (2, 4), (4, 4)]]

In [15]:
rdd_grouped = rdd_sample.groupByKey()# wide transformation

In [16]:
rdd_grouped

PythonRDD[15] at RDD at PythonRDD.scala:53

In [18]:
for item in rdd_grouped.collect():
    print(item[0], [value for value in item[1]])

3 [5, 4]
1 [4]
4 [4]
2 [2, 1, 5, 10, 7, 1, 4]


In [19]:
rdd_grouped.glom().collect()

[[(3, <pyspark.resultiterable.ResultIterable at 0x7c224254d360>)],
 [(1, <pyspark.resultiterable.ResultIterable at 0x7c224254c640>),
  (4, <pyspark.resultiterable.ResultIterable at 0x7c224254d1e0>)],
 [(2, <pyspark.resultiterable.ResultIterable at 0x7c224254c9d0>)]]

In [20]:
rdd_grouped.getNumPartitions()

3

In [21]:
for item in rdd_grouped.collect():
    print(item[0], [value for value in item[1]])

3 [5, 4]
1 [4]
4 [4]
2 [2, 1, 5, 10, 7, 1, 4]


In [24]:
import numpy as np
import random
key_1 = ['a']* 10
key_2 = ['b']* 6000000
key_3 = ['c']* 800
key_4 = ['d'] * 10000


keys = key_1+key_2+key_3+key_4
random.shuffle(keys)
values_1  = list(np.random.randint(low = 1,high=100,size = len(key_1)))
values_2  = list(np.random.randint(low = 1,high=100,size = len(key_2)))
values_3  = list(np.random.randint(low = 1,high=100,size = len(key_3)))
values_4  = list(np.random.randint(low = 1,high=100,size = len(key_4)))
values = values_1+values_2+values_3+values_4

pair_skew = list(zip(keys,values))

In [25]:
# load data into rdd
rdd = sc.parallelize(pair_skew,8)

In [26]:
group_rdd_data = rdd.groupByKey().cache()# chache 
group_rdd_data.map(lambda pair:(pair[0],[i+10 for i in pair[1]])).count()

24/04/10 21:08:20 WARN TaskSetManager: Stage 13 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.


4

In [27]:
# salting
def salting(val):
    tmp = val+"_"+str(random.randint(0,5))
    return tmp

In [29]:
#salted_rdd
salted_rdd = rdd.map(lambda x:(salting(x[0]),x[1]))
grouped_rdd_salted = salted_rdd.groupByKey().cache()
grouped_rdd_salted.map(lambda pair:(pair[0],[i+10 for i in pair[1]])).count()

24/04/10 21:23:14 WARN TaskSetManager: Stage 15 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.


23

In [30]:
rdd_sort = rdd.sortByKey(ascending = False,numPartitions=4)
rdd_sort.count()

24/04/10 21:34:45 WARN TaskSetManager: Stage 17 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.
24/04/10 21:34:49 WARN TaskSetManager: Stage 18 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.
24/04/10 21:34:55 WARN TaskSetManager: Stage 19 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.


6010810

In [31]:
rdd_sort = salted_rdd.sortByKey(ascending = False,numPartitions=4)
rdd_sort.count()

24/04/10 21:37:27 WARN TaskSetManager: Stage 21 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.
24/04/10 21:37:41 WARN TaskSetManager: Stage 22 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.
24/04/10 21:37:54 WARN TaskSetManager: Stage 23 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.


6010810

In [32]:
# join 


rdd1 = sc.parallelize([(2,3),(1,3),(1,4),(3,1),(5,1)],3)
rdd2 = sc.parallelize([(4,3),(0,1),(1,2),(2,1)],2)
join1 = rdd1.join(rdd2)
join1.collect()

[(1, (3, 2)), (1, (4, 2)), (2, (3, 1))]

In [33]:
join1.getNumPartitions()

5

In [34]:
join1.glom().collect()

[[], [(1, (3, 2)), (1, (4, 2))], [(2, (3, 1))], [], []]

In [36]:
# create sample data for join

import numpy as np
import random
key_1 = ['a']* 5
key_2 = ['b']* 60
key_3 = ['c']* 100



keys = key_1+key_2+key_3
random.shuffle(keys)
values_1  = list(np.random.randint(low = 1,high=100,size = len(key_1)))
values_2  = list(np.random.randint(low = 1,high=100,size = len(key_2)))
values_3  = list(np.random.randint(low = 1,high=100,size = len(key_3)))

values = values_1+values_2+values_3

pair_data = list(zip(keys,values))
rdd_small = sc.parallelize(pair_data,2)
# join without salting

rdd_join_noSalting = rdd.join(rdd_small)
rdd_join_noSalting.map(lambda x:x[1][0]+x[1][1]).reduce(lambda a,b:a+b)

24/04/10 22:10:07 WARN TaskSetManager: Stage 29 contains a task of very large size (16951 KiB). The maximum recommended task size is 1000 KiB.


35600986935

In [37]:
rdd_join_noSalting.getNumPartitions()

10